In [1]:
# DFS running combat

A series refers to the sequence of subactions and relevant details needed by the process_turn function.

A full_list refers to the collection of series options an actor has to pick from.

In [2]:
import multiprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import csv

In [3]:
from DFS_World_States import world, world_grid_states, Weapon, Potion, Shield, Wand
from DFS_Universal_Rules import subaction_dict, target_distance_scores, fire_bolt, ray_of_frost, shield
from DFS_Action_Series import RuleBasedSequenceDFS, action_rules_permit, action_rules_sources, RuleBasedSequenceDFS2
from DFS_Location_Series import RuleBasedLocationSequenceDFS2, location_rules, location_rules_sources, RuleBasedLocationSequenceDFS3, location_rules_permit
from DFS_Functions import analyze_reward_distribution, analyze_reward_distribution_series

from DFS_Object_Series import RuleBasedObjectSequenceDFS1, object_rules, object_rules_sources
from DFS_Entity_Series import RuleBasedEntitySequenceDFS, entity_rules, entity_rules_sources

In [4]:
from DFS_Entities import entity
from DFS_Reward_Calculators import check_rules_for_reward, calc_new_reward, precalc_reward, action_rules_reward, precalc_reward_series, post_loc_series_reward_calc, post_obj_reward_series_calc2, action_location_reward_rules_reward

In [5]:
stage = world(11)

stage.generate_map()

actor = entity('player character',stage,None,'medium')
stage.add_entity([0,0],actor)

actor.subactions_based_on_move_speed()
actor.subactions_post_move_speed()
actor.subactions_universal()
actor.subactions_object_interactions()
actor.subactions_attack()
actor.subactions_teamwork()
actor.subactions_casting()
actor.subactions_escape()

actor.add_rules_for_dfs(action_rules_permit, 'action_rules', 'permit')
actor.add_rules_for_dfs(action_rules_reward, 'action_rules', 'reward')
actor.add_rules_for_dfs(location_rules_permit, 'location_rules', 'permit')
actor.add_rules_for_dfs(action_location_reward_rules_reward, 'location_rules', 'reward')


actor.print_subactions_and_catalog()

actor.add_spell(fire_bolt)
actor.add_spell(ray_of_frost)
actor.add_spell(shield)

stage.add_item_to_inventory(actor, Weapon('dagger1', 'weapon', 1, 1, 0))
stage.add_item_to_inventory(actor, Weapon('dagger2', 'weapon', 1, 1, 0))
stage.add_item_to_inventory(actor, Weapon('greataxe', 'weapon', 1, 1, 0))
stage.add_item_to_inventory(actor, Shield('shield','shield', 1))
stage.add_item_to_inventory(actor, Wand('wand','wand', 1))



monster_one = entity('monster',stage,None,'small','goblin','1x1',False,6)
stage.add_entity([2,2],monster_one)

monster_two = entity('monster',stage,None,'medium','guard','1x1',False,6)
stage.add_entity([1,1],monster_two)

monster_three = entity('monster',stage,None,'large','horse','2x2',True,12)
stage.add_entity([0,-3],monster_three)


#stage.add_enemy((-2,0))
#stage.add_enemy((0,3))
#stage.add_enemy((2,2))
#stage.add_enemy((-3,-3))
#stage.add_enemy((1,-1))

stage.add_coin((3,0))
stage.add_coin((-1,-1))
stage.add_coin((1,1))




stage.add_weapon_dagger((-2,-1))
stage.add_weapon_dagger((-1,-2))
stage.add_weapon_greataxe((2,0))


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]

move_subactions_text ['move 1 space', 'move 2 space', 'move 3 space', 'move 4 space', 'move 5 space', 'move 6 space', 'go prone', 'stand up', 'mount', 'dismount']
move_subactions_sources [['basic rules', 'speed'], ['basic rules', 'speed'], ['basic rules', 'speed'], ['basic rules', 'speed'], ['basic rules', 'speed'], ['basic rules', 'speed'], ['basic rules'], ['basic rules', 'speed'], ['basic rules', 'speed'], ['basic rules', 'speed']]
move_subactions_reps [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
action_subactions_text ['dodge', 'disengage', 'hide', 'search', 'object--action--ground_pickup', 'object--action--ground_drop_from_inventory', 'object--action--ground_drop_main_hand', 'object--action--ground_drop_off_hand', 'object--action--ground_drop_both_hands', 'object--action--self_equip_mai

In [6]:
#stage.grid2

In [7]:
# I should display a visual here of the grid and where each entity and object is located

# stage.grid2 is a inhomogeneous list of lists of lists, ints, and strings that represent the grid

# I only want to see the entities and objects, so I will create a new list of lists that only contains the entities and objects




In [8]:
#action_sequence_generator = RuleBasedSequenceDFS(min_length=0,max_length=theoretical_turn_length, start=0, end=len(subaction_dict), rules=action_rules, acting_entity=actor)
#action_series_full_list = action_sequence_generator.generate_sequences()
#action_series_full_list

In [9]:
#action_sequence_generator = RuleBasedSequenceDFS2(acting_entity=actor)
#action_series_full_list = action_sequence_generator.generate_sequences()

# I want to export this to a csv file so that if I run this code again, and the parameters are the same, I can just import the csv file instead of running the code again
# need to store the parameters used so that I can check if they are the same as the current parameters
#parameters = {'min_length':0,'max_length':theoretical_turn_length,'start':0,'end':len(subaction_dict),'rules':action_rules_permit,'acting_entity':actor}

#action_series_full_list.to_csv('action_series_full_list.csv')
#parameters.to_csv('parameters.csv')



In [10]:
# create an if statement that checks if the parameters are the same as the current parameters
# if they are the same, import the csv files
parameters = {'min_length':1,
              'max_length': len(actor.subaction_catalog['free_subaction_groups']) + actor.speed + 1 + 1,
              'start':0,
              'end':len(actor.subactions),
              'rules':f'{[x for x in actor.dfs_rules['action_rules']['permit']]}',
              'acting_entity':actor
              }



try:
    imported_parameters = pd.read_csv('parameters.csv')
except:
    imported_parameters = None

if parameters == imported_parameters:
    action_series_full_list = pd.read_csv('action_series_full_list.csv')
    
else:
    action_sequence_generator = RuleBasedSequenceDFS2(acting_entity=actor)
    action_series_full_list = action_sequence_generator.generate_sequences()

    # export the action_series_full_list and parameters to csv files
    with open('action_series_full_list.csv', 'w') as f:
        write = csv.writer(f)
        write.writerows(action_series_full_list)

    with open('parameters.csv', 'w') as f:
        write = csv.DictWriter(f, fieldnames=parameters.keys())
        write.writerows(parameters)


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
parameters

In [ ]:
action_series_full_list

In [15]:
#reward_series_full_list = precalc_reward_series(action_series_full_list, actor)
#reward_series_full_list

In [16]:
action_reward_series_full_list = calc_new_reward(action_series_full_list, actor, 'action')

In [ ]:
action_reward_series_full_list

In [ ]:
#analyze_reward_distribution(reward_series_full_list, action_series_full_list)
#action_reward_series_full_list = [[action_series_full_list[i], reward_series_full_list[i]] for i in range(len(action_series_full_list))]

analyze_reward_distribution_series(action_reward_series_full_list, actor, 99)

In [19]:
#location_series_generator = RuleBasedLocationSequenceDFS2(action_series_list=action_series_full_list,
#                                                             reward_series_list=reward_series_full_list,
#                                                             location_rules=location_rules,
#                                                             acting_entity=actor,
#                                                             target_distance_scores=target_distance_scores)
#location_series_full_list = location_series_generator.generate_sequences()

In [ ]:
location_series_generator = RuleBasedLocationSequenceDFS3(acting_entity=actor,
                                                          action_reward_series_full_list=action_reward_series_full_list)
action_location_series_full_list = location_series_generator.generate_sequences()

In [ ]:
action_location_series_full_list

In [18]:
action_location_reward_series_full_list = calc_new_reward(action_location_series_full_list, actor, 'action_location_reward')

In [ ]:
action_location_reward_series_full_list

In [ ]:
analyze_reward_distribution_series(action_location_reward_series_full_list, actor, 99)

In [ ]:
#post_location_reward_list = post_loc_series_reward_calc(action_series_full_list, location_series_full_list, reward_series_full_list, actor)

In [ ]:
analyze_reward_distribution_series(post_location_reward_list, actor, 99)

In [12]:
# Object Series
object_series_generator = RuleBasedObjectSequenceDFS1(post_location_reward_list = post_location_reward_list, 
                                                     target_distance_scores = target_distance_scores,
                                                     object_rules = object_rules, 
                                                     acting_entity = actor)
object_series_full_list = object_series_generator.generate_object_sequences()

In [ ]:
post_object_reward_list = post_obj_reward_series_calc2(object_series_full_list, actor)
post_object_reward_list

In [ ]:
analyze_reward_distribution_series(post_object_reward_list, actor, 99)

In [15]:
# entity series
# in order to specify which entity is being targeted

# shortcuts:
# - currently based on the location series, if there is only one entity in a space, the entity is automatically targeted
# - only if there are multiple entities in a space, the entity series is used to determine the target

In [ ]:
entity_series_generator = RuleBasedEntitySequenceDFS(action_location_object_reward_series_full_list = post_object_reward_list,
                                                     target_distance_scores = target_distance_scores,
                                                     entity_rules = entity_rules,
                                                     acting_entity = actor)
entity_series_generator.generate_entity_sequences()

In [17]:
action_location_object_entity_reward_series_full_list = entity_series_generator.generate_entity_sequences()

In [ ]:
action_location_object_entity_reward_series_full_list

In [ ]:
action_location_object_entity_reward_series_full_list = calc_new_reward(action_location_object_entity_reward_series_full_list, actor)

In [ ]:
analyze_reward_distribution_series(action_location_object_entity_reward_series_full_list, actor, 99)

In [ ]:
# spell series


In [24]:
#post_object_reward_list = post_obj_reward_series_calc(action_series_full_list = action_series_full_list, 
#                                                      location_series_full_list = location_series_full_list, 
#                                                      object_series_full_list = object_series_full_list, 
#                                                      reward_series_full_list = reward_series_full_list, 
#                                                      acting_entity = actor)

# this one will include how much damage is being dealt
# if an effective weapon is equipped
# how many coins are picked up

In [26]:
# processing a round of combat that includes the queue of actions and possible reactions other entities take

In [27]:
# what needs to happen before I can be running combat?
# - the function to process a creature's turn needs to be done
# - the function to process everyone else's reaction needs to be operational
# - enemies need to be able to take their turn, currently they operate as a form of static entity



In [28]:
# what needs to happen before the next stage?
# - combat situations needs to be randomly generated
# - combat scenarios need to be ran millions of times